In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/jeffshamp/.conda/envs/sps620/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df=pd.read_csv('./data/global_terror_data_proj_1.csv')

In [3]:
ter_df = df[["country_txt", "attacktype1", "targtype1"]]
ter_df.head()

KeyError: "['targtype1'] not in index"

In [22]:
G=nx.from_pandas_dataframe(ter_df, "country_txt", "attacktype1", "targtype1")
print (nx.info(G))

Name: 
Type: Graph
Number of nodes: 204
Number of edges: 1226
Average degree:  12.0196


In [37]:
from pyvis import network as net
N = net.Network(
                bgcolor='#222222', font_color='white', 
               notebook=True)
N.barnes_hut()
for n in G.nodes():
    N.add_node(n)
for e in G.edges():
    N.add_edge(e[0], e[1])

In [38]:
N.show('test.html')

In [51]:
ter_df_2 = df[["country_txt", "gname","attacktype1"]]

In [54]:
H=nx.from_pandas_dataframe(ter_df_2, "country_txt", "gname","attacktype1")
N_2 = net.Network(bgcolor='#222222',
                  font_color='white',
                  height='1000px',
                  width='1000px',
                  notebook=True)
N_2.barnes_hut()
for n in H.nodes():
    N_2.add_node(n)
for e in H.edges():
    N_2.add_edge(e[0], e[1])

In [55]:
N_2.show('test_2.html')